#### Pandas Tutorial - Part 38

This notebook covers:
- Advanced Excel file operations
- Working with other data formats
- Connecting to databases and cloud services

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os

%matplotlib inline

##### Advanced Excel File Operations

Pandas provides extensive functionality for working with Excel files through the `read_excel()` and `to_excel()` functions.

### Reading Excel Files with Different Options

The `read_excel()` function offers many parameters to customize how data is read from Excel files.

In [3]:
# Create a sample DataFrame to write to Excel
df_excel = pd.DataFrame({
    'Name': ['John', 'Jane', 'Bob', 'Alice', 'Mike'],
    'Age': [28, 34, 42, 31, 37],
    'Department': ['HR', 'Engineering', 'Marketing', 'Finance', 'IT'],
    'Salary': [50000, 60000, 55000, 65000, 58000]
})

# Create another DataFrame for a second sheet
df_excel2 = pd.DataFrame({
    'Product': ['A', 'B', 'C', 'D'],
    'Price': [100, 200, 150, 300],
    'Quantity': [10, 15, 8, 12]
})

# Write both DataFrames to different sheets in the same Excel file
with pd.ExcelWriter('sample_multi_sheet.xlsx') as writer:
    df_excel.to_excel(writer, sheet_name='Employees', index=False)
    df_excel2.to_excel(writer, sheet_name='Products', index=False)

print("Excel file with multiple sheets created.")

Excel file with multiple sheets created.


#### Reading Specific Sheets

In [4]:
# Read the first sheet (default)
df1 = pd.read_excel('sample_multi_sheet.xlsx')
print("First sheet (default):")
df1

First sheet (default):


,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


In [5]:
# Read a specific sheet by name
df2 = pd.read_excel('sample_multi_sheet.xlsx', sheet_name='Products')
print("Products sheet:")
df2

Products sheet:


,Product,Price,Quantity
0,A,100,10
1,B,200,15
2,C,150,8
3,D,300,12


In [6]:
# Read multiple sheets
dfs = pd.read_excel('sample_multi_sheet.xlsx', sheet_name=['Employees', 'Products'])
print("Type of result when reading multiple sheets:", type(dfs))
print("\nKeys (sheet names):", list(dfs.keys()))
print("\nEmployees sheet:")
dfs['Employees'].head()

Type of result when reading multiple sheets: <class 'dict'>

Keys (sheet names): ['Employees', 'Products']

Employees sheet:


,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


In [7]:
# Read all sheets
all_dfs = pd.read_excel('sample_multi_sheet.xlsx', sheet_name=None)
print("All sheets:", list(all_dfs.keys()))

All sheets: ['Employees', 'Products']


#### Selecting Specific Columns

In [8]:
# Read only specific columns
df_cols = pd.read_excel('sample_multi_sheet.xlsx', usecols=['Name', 'Salary'])
df_cols

,Name,Salary
0,John,50000
1,Jane,60000
2,Bob,55000
3,Alice,65000
4,Mike,58000


In [9]:
# Read columns by position
df_cols_pos = pd.read_excel('sample_multi_sheet.xlsx', usecols=[0, 3])  # First and fourth columns
df_cols_pos

,Name,Salary
0,John,50000
1,Jane,60000
2,Bob,55000
3,Alice,65000
4,Mike,58000


In [10]:
# Read columns using a callable
df_cols_callable = pd.read_excel('sample_multi_sheet.xlsx', 
                                 usecols=lambda x: 'a' in x.lower())  # Columns with 'a' in the name
df_cols_callable

,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


#### Setting Index Columns and Headers

In [11]:
# Use a column as the index
df_index = pd.read_excel('sample_multi_sheet.xlsx', index_col='Name')
df_index

,Age,Department,Salary
Name,,,
John,28,HR,50000
Jane,34,Engineering,60000
Bob,42,Marketing,55000
Alice,31,Finance,65000
Mike,37,IT,58000


In [12]:
# Create a file with no header for demonstration
df_no_header = pd.DataFrame([
    ['John', 28, 'HR', 50000],
    ['Jane', 34, 'Engineering', 60000],
    ['Bob', 42, 'Marketing', 55000]
])
df_no_header.to_excel('no_header.xlsx', index=False, header=False)

# Read file with no header, providing column names
df_custom_header = pd.read_excel('no_header.xlsx', header=None, 
                                 names=['Name', 'Age', 'Department', 'Salary'])
df_custom_header

,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000


### Working with ExcelFile Objects

For more control over Excel file operations, you can use the `ExcelFile` class.

In [13]:
# Create an ExcelFile object
excel_file = pd.ExcelFile('sample_multi_sheet.xlsx')

# List all sheet names
print("Sheet names:", excel_file.sheet_names)

# Parse a specific sheet
df_parsed = excel_file.parse('Employees')
df_parsed.head()

Sheet names: ['Employees', 'Products']


,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


In [14]:
# Parse with additional options
df_parsed_options = excel_file.parse('Employees', usecols=['Name', 'Age'])
df_parsed_options

,Name,Age
0,John,28
1,Jane,34
2,Bob,42
3,Alice,31
4,Mike,37


##### Working with Other Data Formats

Pandas supports many other data formats beyond CSV and Excel.

### JSON Data

JSON (JavaScript Object Notation) is a common format for web APIs and configuration files.

In [15]:
# Create a sample DataFrame
df_json = pd.DataFrame({
    'Name': ['John', 'Jane', 'Bob'],
    'Age': [28, 34, 42],
    'Scores': [[85, 90, 78], [92, 88, 95], [75, 80, 85]]
})
df_json

,Name,Age,Scores
0,John,28,"[85, 90, 78]"
1,Jane,34,"[92, 88, 95]"
2,Bob,42,"[75, 80, 85]"


In [16]:
# Write to JSON
df_json.to_json('sample.json')
print("JSON file created.")

# Read the JSON file
df_from_json = pd.read_json('sample.json')
df_from_json

JSON file created.


,Name,Age,Scores
0,John,28,"[85, 90, 78]"
1,Jane,34,"[92, 88, 95]"
2,Bob,42,"[75, 80, 85]"


In [17]:
# Write to JSON with different orientations
print("Default orientation:")
print(df_json.to_json())

print("\nRecords orientation:")
print(df_json.to_json(orient='records'))

print("\nTable orientation:")
print(df_json.to_json(orient='table'))

Default orientation:
{"Name":{"0":"John","1":"Jane","2":"Bob"},"Age":{"0":28,"1":34,"2":42},"Scores":{"0":[85,90,78],"1":[92,88,95],"2":[75,80,85]}}

Records orientation:
[{"Name":"John","Age":28,"Scores":[85,90,78]},{"Name":"Jane","Age":34,"Scores":[92,88,95]},{"Name":"Bob","Age":42,"Scores":[75,80,85]}]

Table orientation:
{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"Name","type":"string"},{"name":"Age","type":"integer"},{"name":"Scores","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":0,"Name":"John","Age":28,"Scores":[85,90,78]},{"index":1,"Name":"Jane","Age":34,"Scores":[92,88,95]},{"index":2,"Name":"Bob","Age":42,"Scores":[75,80,85]}]}


### HTML Data

Pandas can read tables from HTML pages and write DataFrames as HTML tables.

In [19]:
# Write DataFrame to HTML
html = df_excel.to_html()
with open('sample.html', 'w') as f:
    f.write(html)
print("HTML file created.")

# Read HTML tables from a file
tables = pd.read_html('sample.html')
print(f"Number of tables found: {len(tables)}")
tables[0].head()

HTML file created.
Number of tables found: 1


,Unnamed: 0,Name,Age,Department,Salary
0,0,John,28,HR,50000
1,1,Jane,34,Engineering,60000
2,2,Bob,42,Marketing,55000
3,3,Alice,31,Finance,65000
4,4,Mike,37,IT,58000


### Pickle Files

Pickle is a Python-specific binary format that can store complex objects.

In [20]:
# Write to pickle
df_excel.to_pickle('sample.pkl')
print("Pickle file created.")

# Read from pickle
df_from_pickle = pd.read_pickle('sample.pkl')
df_from_pickle

Pickle file created.


,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


##### Connecting to Databases and Cloud Services

Pandas can connect to various databases and cloud services to read and write data.

### SQL Databases

Pandas can interact with SQL databases using SQLAlchemy.

In [21]:
# Example of reading from a SQLite database (commented out as it requires a database)
"""
import sqlite3

# Create a connection to a SQLite database
conn = sqlite3.connect('example.db')

# Create a table and insert data
df_excel.to_sql('employees', conn, if_exists='replace', index=False)

# Read data from the database
df_from_sql = pd.read_sql('SELECT * FROM employees', conn)
df_from_sql
"""

"\nimport sqlite3\n\n# Create a connection to a SQLite database\nconn = sqlite3.connect('example.db')\n\n# Create a table and insert data\ndf_excel.to_sql('employees', conn, if_exists='replace', index=False)\n\n# Read data from the database\ndf_from_sql = pd.read_sql('SELECT * FROM employees', conn)\ndf_from_sql\n"

### Google BigQuery

Pandas can read data from Google BigQuery using the `pandas-gbq` package.

In [22]:
# Example of reading from Google BigQuery (commented out as it requires authentication)
"""
# This requires the pandas-gbq package
# !pip install pandas-gbq

# Read data from BigQuery
df_gbq = pd.read_gbq(
    'SELECT * FROM `bigquery-public-data.samples.shakespeare` LIMIT 10',
    project_id='your-project-id'
)
df_gbq
"""

"\n# This requires the pandas-gbq package\n# !pip install pandas-gbq\n\n# Read data from BigQuery\ndf_gbq = pd.read_gbq(\n    'SELECT * FROM `bigquery-public-data.samples.shakespeare` LIMIT 10',\n    project_id='your-project-id'\n)\ndf_gbq\n"

### HDF5 Files

HDF5 is a high-performance data format for storing large amounts of data.

In [24]:
# Write to HDF5
df_excel.to_hdf('sample.h5', key='employees', mode='w')
print("HDF5 file created.")

# Read from HDF5
df_from_hdf = pd.read_hdf('sample.h5', 'employees')
df_from_hdf

HDF5 file created.


,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


### Parquet Files

Parquet is a columnar storage format that is highly efficient for analytical queries.

In [26]:
# Write to Parquet
df_excel.to_parquet('sample.parquet')
print("Parquet file created.")

# Read from Parquet
df_from_parquet = pd.read_parquet('sample.parquet')
df_from_parquet

Parquet file created.


,Name,Age,Department,Salary
0,John,28,HR,50000
1,Jane,34,Engineering,60000
2,Bob,42,Marketing,55000
3,Alice,31,Finance,65000
4,Mike,37,IT,58000


##### Conclusion

In this notebook, we've explored:

1. Advanced Excel file operations, including:
   - Reading specific sheets
   - Selecting specific columns
   - Setting index columns and headers
   - Working with ExcelFile objects

2. Working with other data formats, including:
   - JSON data
   - HTML data
   - Pickle files

3. Connecting to databases and cloud services, including:
   - SQL databases
   - Google BigQuery
   - HDF5 files
   - Parquet files

These capabilities make pandas a versatile tool for working with data from various sources and formats, allowing you to integrate data from different systems into your analysis workflow.